In [2]:
# train.py
import wandb
import random  # for demo script

epochs = 10
lr = 0.01
run = wandb.init(
    # Set the project where this run will be logged
    project="KOREA-basic-logging",
    name="basic-logging",
    # Track hyperparameters and run metadata
    config={
        "learning_rate": lr,
        "epochs": epochs,
    },
)

offset = random.random() / 5
print(f"lr: {lr}")

# simulating a training run
for epoch in range(2, epochs):
    acc = 1 - 2**-epoch - random.random() / epoch - offset
    loss = 2**-epoch + random.random() / epoch + offset
    print(f"epoch={epoch}, accuracy={acc}, loss={loss}")
    wandb.log({"accuracy": acc, "loss": loss})

run.finish()

lr: 0.01
epoch=2, accuracy=0.34028766845022473, loss=0.5284515804636171
epoch=3, accuracy=0.45853171093146644, loss=0.24365474040693458
epoch=4, accuracy=0.653815992076043, loss=0.30025269175326014
epoch=5, accuracy=0.8136005437313486, loss=0.18829476903173425
epoch=6, accuracy=0.7927711211056596, loss=0.23676577876421473
epoch=7, accuracy=0.7879827289369249, loss=0.22003520164253781
epoch=8, accuracy=0.8201413969062276, loss=0.1299559670788395
epoch=9, accuracy=0.8136503308022457, loss=0.15297800999151054


accuracy,▁▃▆█████
loss,█▃▄▂▃▃▁▁
accuracy,0.81365
loss,0.15298


# fashion-mnist

In [5]:
import wandb
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor
import wandb

class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28 * 28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10)
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits


def train(dataloader, model, loss_fn, optimizer, device):
    size = len(dataloader.dataset)
    model.train()
    total_loss = 0
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)

        # Compute prediction error
        pred = model(X)
        loss = loss_fn(pred, y)
        total_loss += loss.item()

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")
    return model, total_loss


def test(dataloader, model, loss_fn, device):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")
    return correct, test_loss

In [35]:
import numpy as np

PROJECT = "KOREA-basic-logging" 
CONFIG = {
    "learning_rate": 1e-3,
    "epochs": 30,
    "batch_size": 64
}

## Download training data from open datasets.
training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor(),
)

test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor(),
)

with wandb.init(project=PROJECT, name='fashion_mnist') as run:
    wandb.config.update(CONFIG)
    
    train_dataloader = DataLoader(training_data, batch_size=CONFIG['batch_size'])
    test_dataloader = DataLoader(test_data, batch_size=CONFIG['batch_size'])
    device = "cuda" if torch.cuda.is_available() else "cpu"
    model = NeuralNetwork().to(device)
    loss_fn = nn.CrossEntropyLoss()
    optimizer = torch.optim.SGD(model.parameters(), lr=CONFIG['learning_rate'])
    epochs = CONFIG['epochs']
    for t in range(epochs):
        print(f"Epoch {t+1}\n-------------------------------")
        model, total_loss = train(train_dataloader, model, loss_fn, optimizer, device)
        correct, test_loss = test(test_dataloader, model, loss_fn, device)
        #####################################################################
        wandb.log({"train_loss": total_loss / len(train_dataloader)}, step=t)
        wandb.log({"test_loss": test_loss, "test_acc": correct}, step=t)
        #####################################################################

    torch.save(model.state_dict(), "model.pth")
    print("Saved PyTorch Model State to model.pth")

    classes = [
        "T-shirt/top",
        "Trouser",
        "Pullover",
        "Dress",
        "Coat",
        "Sandal",
        "Shirt",
        "Sneaker",
        "Bag",
        "Ankle boot",
    ]

    model.eval()
    #####################################################################
    table = wandb.Table(columns=["Image", "True", "Pred"]+["score_"+str(cls) for cls in classes])
    for i in range(100):
        x, y = test_data[i][0], test_data[i][1]
        with torch.no_grad():
            pred = model(x)
            predicted, actual = classes[pred[0].argmax(0)], classes[y]
            row = [wandb.Image(x), actual, predicted]
        for s in nn.functional.softmax(pred[0], dim=0):
            row.append(np.round(s.item(), 4))
        table.add_data(*row)
    wandb.log({"results": table})

    model_artifact = wandb.Artifact("SimpleNN", type="model")
    model_artifact.add_file("model.pth")
    run.log_artifact(model_artifact)
    #####################################################################

Epoch 1
-------------------------------
loss: 2.306665  [    0/60000]
loss: 2.296305  [ 6400/60000]
loss: 2.285831  [12800/60000]
loss: 2.275164  [19200/60000]
loss: 2.267049  [25600/60000]
loss: 2.232993  [32000/60000]
loss: 2.242207  [38400/60000]
loss: 2.209112  [44800/60000]
loss: 2.208767  [51200/60000]
loss: 2.169536  [57600/60000]
Test Error: 
 Accuracy: 38.7%, Avg loss: 2.174053 

Saved PyTorch Model State to model.pth


test_acc,▁
test_loss,▁
train_loss,▁
test_acc,0.3866
test_loss,2.17405
train_loss,2.24547
